# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...


## Generation Step

Python coding capabilities of the openai/gpt-oss-20b model. 
In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv(dotenv_path="C:\\Users\\dongr\\Desktop\\Agentic AI_Intern\\.env")


client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the code.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama-3.3-70b-versatile"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

**Merge Sort Algorithm Implementation in Python**
=====================================================

### Introduction

Merge Sort is a divide-and-conquer algorithm that splits an input array into two halves, recursively sorts each half, and then merges them.

### Code

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Args:
    arr (list): The input array to be sorted.

    Returns:
    list: The sorted array.
    """
    # Base case: If the array has 1 or fewer elements, it is already sorted.
    if len(arr) <= 1:
        return arr

    # Find the middle index to split the array into two halves.
    mid = len(arr) // 2

    # Recursively sort the left and right halves.
    left_half = merge_sort(arr[:mid])
    right_half = merge_sort(arr[mid:])

    # Merge the sorted halves.
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    Args:
    left (list): The first sorted array.
    right (list): The second sorted array.

    Returns:
    list: The merged sorted array.
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first.
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Append any remaining elements.
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage
if __name__ == "__main__":
    arr = [64, 34, 25, 12, 22, 11, 90]
    print("Original array:", arr)
    print("Sorted array:", merge_sort(arr))

```

### Explanation

1.  The `merge_sort` function takes an input array and recursively splits it into two halves until each sub-array has one or fewer elements (base case).
2.  The `merge` function merges two sorted arrays into a single sorted array by comparing elements and placing the smaller one first.
3.  The `merge_sort` function uses the `merge` function to merge the sorted halves and returns the final sorted array.

### Time Complexity

*   Best-case: O(n log n)
*   Average-case: O(n log n)
*   Worst-case: O(n log n)

### Space Complexity

*   O(n) auxiliary space for the recursive call stack and temporary arrays.

This implementation provides a clear and efficient implementation of the Merge Sort algorithm in Python.

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Hadley Wickham a computer scientist and Deep Learning wizard.


In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Hadley Wickham, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama-3.3-70b-versatile"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

## Code Review and Recommendations
### Introduction

The provided implementation of the Merge Sort algorithm is well-structured, readable, and efficient. However, there are some improvements that can be made to enhance the code's performance, readability, and maintainability.

### Code Quality and Best Practices

*   **Type Hints**: Add type hints for function parameters and return types to improve code readability and enable better type checking.
*   **Docstrings**: While the code has docstrings, they can be more detailed and include information about the time and space complexity of each function.
*   **Variable Names**: Variable names like `arr`, `left`, and `right` are not very descriptive. Consider using more descriptive names like `input_array`, `left_half`, and `right_half`.

### Performance Improvements

*   **In-Place Merge**: The current implementation creates new lists for the merged array. Consider implementing an in-place merge to reduce memory allocation and deallocation overhead.
*   **Iterative Merge Sort**: While the recursive approach is intuitive, an iterative implementation can be more efficient for large inputs due to the overhead of recursive function calls.

### Code Refactoring

Here's an updated version of the code with the suggested improvements:

```python
def merge_sort(input_array: list) -> list:
    """
    Sorts an input array using the Merge Sort algorithm.

    Args:
    input_array (list): The input array to be sorted.

    Returns:
    list: The sorted array.

    Time Complexity:
    O(n log n)

    Space Complexity:
    O(n) auxiliary space for the recursive call stack and temporary arrays.
    """
    # Base case: If the array has 1 or fewer elements, it is already sorted.
    if len(input_array) <= 1:
        return input_array

    # Find the middle index to split the array into two halves.
    mid = len(input_array) // 2

    # Recursively sort the left and right halves.
    left_half = merge_sort(input_array[:mid])
    right_half = merge_sort(input_array[mid:])

    # Merge the sorted halves.
    return merge(left_half, right_half)


def merge(left_half: list, right_half: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    Args:
    left_half (list): The first sorted array.
    right_half (list): The second sorted array.

    Returns:
    list: The merged sorted array.

    Time Complexity:
    O(n)

    Space Complexity:
    O(n) auxiliary space for the merged array.
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first.
    while left_index < len(left_half) and right_index < len(right_half):
        if left_half[left_index] <= right_half[right_index]:
            merged.append(left_half[left_index])
            left_index += 1
        else:
            merged.append(right_half[right_index])
            right_index += 1

    # Append any remaining elements.
    merged.extend(left_half[left_index:])
    merged.extend(right_half[right_index:])

    return merged


# Example usage
if __name__ == "__main__":
    input_array = [64, 34, 25, 12, 22, 11, 90]
    print("Original array:", input_array)
    print("Sorted array:", merge_sort(input_array))

```

### Iterative Merge Sort Implementation

Here's an example of an iterative Merge Sort implementation:

```python
def iterative_merge_sort(input_array: list) -> list:
    """
    Sorts an input array using the iterative Merge Sort algorithm.

    Args:
    input_array (list): The input array to be sorted.

    Returns:
    list: The sorted array.

    Time Complexity:
    O(n log n)

    Space Complexity:
    O(n) auxiliary space for the merged array.
    """
    # Calculate the width of the sub-arrays to merge.
    width = 1
    while width < len(input_array):
        # Merge sub-arrays of the current width.
        for i in range(0, len(input_array), 2 * width):
            left_half = input_array[i:i + width]
            right_half = input_array[i + width:i + 2 * width]
            input_array[i:i + 2 * width] = merge(left_half, right_half)
        # Double the width for the next iteration.
        width *= 2

    return input_array


# Example usage
if __name__ == "__main__":
    input_array = [64, 34, 25, 12, 22, 11, 90]
    print("Original array:", input_array)
    print("Sorted array:", iterative_merge_sort(input_array))

```

### In-Place Merge Sort Implementation

Implementing an in-place Merge Sort is more complex and might not be as efficient as the standard Merge Sort algorithm. However, here's an example of how you could implement it:

```python
def in_place_merge_sort(input_array: list) -> list:
    """
    Sorts an input array in-place using the in-place Merge Sort algorithm.

    Args:
    input_array (list): The input array to be sorted.

    Returns:
    list: The sorted array.

    Time Complexity:
    O(n log n)

    Space Complexity:
    O(log n) auxiliary space for the recursive call stack.
    """
    def merge(left: int, mid: int, right: int) -> None:
        # Create a temporary array to store the merged sub-array.
        temp = [0] * (right - left + 1)
        left_index = left
        right_index = mid + 1
        temp_index = 0

        # Merge smaller elements first.
        while left_index <= mid and right_index <= right:
            if input_array[left_index] <= input_array[right_index]:
                temp[temp_index] = input_array[left_index]
                left_index += 1
            else:
                temp[temp_index] = input_array[right_index]
                right_index += 1
            temp_index += 1

        # Append any remaining elements.
        while left_index <= mid:
            temp[temp_index] = input_array[left_index]
            left_index += 1
            temp_index += 1
        while right_index <= right:
            temp[temp_index] = input_array[right_index]
            right_index += 1
            temp_index += 1

        # Copy the merged sub-array back to the original array.
        for i in range(left, right + 1):
            input_array[i] = temp[i - left]

    def sort(left: int, right: int) -> None:
        if left < right:
            mid = (left + right) // 2
            sort(left, mid)
            sort(mid + 1, right)
            merge(left, mid, right)

    sort(0, len(input_array) - 1)
    return input_array


# Example usage
if __name__ == "__main__":
    input_array = [64, 34, 25, 12, 22, 11, 90]
    print("Original array:", input_array)
    print("Sorted array:", in_place_merge_sort(input_array))

```

Note that the in-place Merge Sort implementation has a higher constant factor due to the extra overhead of creating and managing temporary arrays. Therefore, it may not be the best choice for all scenarios.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama-3.3-70b-versatile"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

## Code Review and Recommendations

### Introduction

The provided implementation of the Merge Sort algorithm has been reviewed and recommendations have been made to improve the code's performance, readability, and maintainability. The current implementation is well-structured and efficient, but there are opportunities to enhance its quality and adhere to best practices.

### Code Quality and Best Practices

*   **Type Hints**: Type hints have been added for function parameters and return types to improve code readability and enable better type checking.
*   **Docstrings**: Docstrings have been updated to provide more detailed information about each function, including their time and space complexity.
*   **Variable Names**: Variable names have been modified to be more descriptive, such as using `input_array` instead of `arr`, and `left_half` instead of `left`.

### Performance Improvements

*   **In-Place Merge**: An in-place merge implementation has been provided to reduce memory allocation and deallocation overhead.
*   **Iterative Merge Sort**: An iterative Merge Sort implementation has been provided as an alternative to the recursive approach, which can be more efficient for large inputs due to the overhead of recursive function calls.

### Code Refactoring

The updated code includes the following improvements:

*   Improved type hints and docstrings for better code readability and maintainability.
*   More descriptive variable names for enhanced code clarity.
*   An iterative Merge Sort implementation for improved performance.
*   An in-place Merge Sort implementation for reduced memory allocation and deallocation overhead.

### Code

The updated code includes the following implementations:

*   **Iterative Merge Sort**: An example of an iterative Merge Sort implementation.
*   **In-Place Merge Sort**: An example of an in-place Merge Sort implementation.

These implementations provide alternatives to the standard Merge Sort algorithm and can be used depending on the specific requirements of the application.

### Example Usage

Example usage of the updated code includes:

*   **Iterative Merge Sort**: An example of how to use the iterative Merge Sort implementation.
*   **In-Place Merge Sort**: An example of how to use the in-place Merge Sort implementation.

These examples demonstrate how to utilize the updated code and its features.

### Time Complexity

The time complexity of the updated code is as follows:

*   **Iterative Merge Sort**: O(n log n)
*   **In-Place Merge Sort**: O(n log n)

The time complexity remains the same as the standard Merge Sort algorithm.

### Space Complexity

The space complexity of the updated code is as follows:

*   **Iterative Merge Sort**: O(n) auxiliary space for the merged array.
*   **In-Place Merge Sort**: O(log n) auxiliary space for the recursive call stack.

The space complexity has been improved in the in-place Merge Sort implementation.

### Conclusion

The updated code includes improvements to the original Merge Sort implementation, such as type hints, docstrings, and more descriptive variable names. Additionally, iterative and in-place Merge Sort implementations have been provided as alternatives to the standard algorithm. These improvements enhance the code's performance, readability, and maintainability, making it more suitable for a variety of applications.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [13]:
from agentic_patterns import ReflectionAgent

In [14]:
agent = ReflectionAgent()

In [15]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [16]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)


STEP 1/10

 

GENERATION

 **Merge Sort Algorithm Implementation in Python**

### Overview

Merge sort is a divide-and-conquer algorithm that splits a list into two halves, recursively sorts each half, and then merges the two sorted halves.

### Code

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    Args:
        arr (list): The input array to be sorted.

    Returns:
        list: The sorted array.
    """
    # Base case: If the array has one or zero elements, it's already sorted
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    Args:
        left (list)

## Final result

In [17]:
from IPython.display import display, Markdown
display(Markdown(final_response))

**Merge Sort Algorithm Implementation in Python**
====================================================

### Overview

Merge sort is a divide-and-conquer algorithm that splits a list into two halves, recursively sorts each half, and then merges the two sorted halves.

### Code

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    Args:
        arr (list): The input array to be sorted.

    Returns:
        list: The sorted array.
    """
    # Base case: If the array has one or zero elements, it's already sorted
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    Args:
        left (list): The first sorted array.
        right (list): The second sorted array.

    Returns:
        list: The merged sorted array.
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Append any remaining elements
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage:
if __name__ == "__main__":
    arr = [64, 34, 25, 12, 22, 11, 90]
    sorted_arr = merge_sort(arr)
    print("Original array:", arr)
    print("Sorted array:", sorted_arr)
```

### Explanation

1.  The `merge_sort` function takes an array as input and recursively divides it into two halves until each sub-array has one or zero elements (base case).
2.  The `merge` function merges two sorted arrays into a single sorted array by comparing elements from each array and appending the smaller element to the merged array.
3.  The sorted sub-arrays are merged back together using the `merge` function, resulting in the final sorted array.

### Time Complexity

*   Best-case time complexity: O(n log n)
*   Average-case time complexity: O(n log n)
*   Worst-case time complexity: O(n log n)

### Space Complexity

*   The space complexity of the merge sort algorithm is O(n), where n is the number of elements in the input array. This is because the algorithm uses additional space to store the merged sub-arrays.